In [ ]:
import pandas as pd 
#pd.set_option('compute.use_numexpr', False)
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip
import yaml
import requests
import csv
import re
from tqdm import tqdm
from itertools import product
import json
import pickle
import matplotlib.pyplot as plt
from itertools import cycle, islice

In [ ]:
import pandas as pd 
#pd.set_option('compute.use_numexpr', False)
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import gzip

import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string

import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

torch.manual_seed(1)

In [ ]:
# # ### Load propublica data
# propub = pd.read_csv( 'data/en-US',low_memory=False) #(28189, 6)

# propub.head(10) #852799 rows × 11 columns

# # import zipfile

# # zf = zipfile.ZipFile('data/en-US.zip') 
# # # if you want to see all files inside zip folder
# # zf.namelist() 
# # # now read your csv file 
# # df = pd.read_csv(zf.open('data/en-US.csv'))


In [ ]:
# ### Load predicted fb all (predicted_fb_all) 0.8 million
predicted_fb_all = pd.read_csv( 'data/predicted_all_para.csv',low_memory=False) #(28189, 6)

predicted_fb_all #852799 rows × 11 columns




In [ ]:
predicted_fb_all.tail(150) 

#Club for Growth Action #C #American Potential Fund#C
#The Lincoln Project #L #Committee to Protect Medicare#L #United We Dream Action#L




## Qualitative analysis using 50 unbias funding entities 

In [ ]:
top_fe = pd.read_csv( 'data/top_50_fe_updated.csv',low_memory=False) #(28189, 6)

In [ ]:
top = top_fe.loc[(top_fe['funding_entity'] == 'American Potential Fund')]#
top
top.to_csv('data/aba.csv', line_terminator='\r\n', encoding='utf-8', index = False) 

In [ ]:


#for accuracy with entity level prediction
count_pp = 0
pb = 0
pt = 0
ab = 0
at = 0
ns = 0
for i in range (0, top_fe.shape[0]):
    if top_fe['funding_entity'][i] == 'American Potential Fund':
        if top_fe.pred_stance[i] == 0:
            pb = pb + 1
        elif top_fe.pred_stance[i] == 1:
            pt = pt + 1
        elif top_fe.pred_stance[i] == 2:
            ab = ab + 1
        elif top_fe.pred_stance[i] == 3:
            at = at + 1
        else:
            ns = ns + 1
        
print(pb , pt, ab, at, ns)   





In [ ]:
### Issue distribution over 2 bias funding entities
fe_issue = predicted_fb_all.loc[(predicted_fb_all['funding_entity'] == 'BIDEN FOR PRESIDENT') | 
                      (predicted_fb_all['funding_entity'] == 'DONALD J. TRUMP FOR PRESIDENT, INC.')]

fe_issue.reset_index(inplace=True) # Resets the index, makes factor a column

fe_issue

In [ ]:
fe_issue['ad_creative_body'].nunique()

In [ ]:
### Issue distribution over 2 non-bias funding entities

fe_issue = top_fe.loc[(top_fe['funding_entity'] == 'Independence USA PAC') | 
                      (top_fe['funding_entity'] == 'American Potential Fund')]


# fe_issue = top_fe.loc[(top_fe['funding_entity'] == 'Union 2020') | 
#                       (top_fe['funding_entity'] == 'America First Action')]




fe_issue.reset_index(inplace=True) # Resets the index, makes factor a column

fe_issue

In [ ]:

ad_id = []
stance_p = []
text = []
fe = []
start = []
stop = []
imp = []
spend = []
region = []
demo = []
issue_p = []
for j in range (0, fe_issue.shape[0] ):
    list_dic = eval(fe_issue['region_distribution'][j])
    #print (list_dic)
    #sys.exit()
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        #if dic['region'] == 'Alabama':
        #if dic['region'] == 'Alabama' or dic['region'] == 'Idaho' or dic['region'] == 'Wyoming': # 3 red
        if dic['region'] == 'California' or dic['region'] == 'New York' or dic['region'] == 'Oregon': # 3 blue
        #if dic['region'] == 'California' or dic['region'] == 'New York' or dic['region'] == 'Illinois' or dic['region'] == 'Minnesota' or dic['region'] == 'Vermont' or dic['region'] == 'Oregon' : #6 blue states
        #if dic['region'] == 'Alabama' or dic['region'] == 'Idaho' or dic['region'] == 'Wyoming' or dic['region'] == 'Kansas' or dic['region'] == 'Arkansas' or dic['region'] == 'Oklahoma' : #6 red states
        #if  dic['region'] == 'Pennsylvania': #'Pennsylvania': #we checked for Alabama(c), Florida(s) in the paper
            ad_id.append(fe_issue.id[j])
            text.append(fe_issue.ad_creative_body[j])
            fe.append(fe_issue.funding_entity[j])
            start.append(fe_issue.ad_delivery_start_time[j])
            stop.append(fe_issue.ad_delivery_stop_time[j])
            imp.append(fe_issue.impressions[j])
            spend.append(fe_issue.spend[j])
            region.append(fe_issue.region_distribution[j])
            demo.append(fe_issue.demographic_distribution[j])
            stance_p.append(fe_issue.pred_stance[j])
            issue_p.append(fe_issue.pred_issue[j])

In [ ]:
state_fe_issue = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'ad_delivery_start_time': start, 'ad_delivery_stop_time' : stop,
                             'impressions' : imp, 'spend' : spend, 'region_distribution': region,
                             'demographic_distribution': demo, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            })
state_fe_issue

In [ ]:

#state_fe_issue['ad_creative_body'].nunique()

#one_state_fe = state_fe_issue.loc[(state_fe_issue['funding_entity'] == 'Independence USA PAC')]
#one_state_fe = state_fe_issue.loc[(state_fe_issue['funding_entity'] == 'Union 2020')]
#one_state_fe


one_state_fe = state_fe_issue.loc[(state_fe_issue['funding_entity'] == 'American Potential Fund')]
one_state_fe

# fe_issue = top_fe.loc[(top_fe['funding_entity'] == 'Independence USA PAC') | 
#                       (top_fe['funding_entity'] == 'American Potential Fund')]



# 
#                       (predicted_fb_all['funding_entity'] == 'DONALD J. TRUMP FOR PRESIDENT, INC.')
# (state_fe_issue['funding_entity'] == 'BIDEN FOR PRESIDENT')


# top_fe.loc[(top_fe['funding_entity'] == 'Union 2020') | 
#                       (top_fe['funding_entity'] == 'America First Action')]



In [ ]:
## This is for Issue distribution over two non-bias and bias funding entities for specific state
sfei = state_fe_issue.groupby(['pred_issue', 'funding_entity']).agg({'id': 'count'}).unstack()
sfei

In [ ]:
## This is for Issue distribution over two non-bias and bias funding entities for all states
fei = fe_issue.groupby(['pred_issue', 'funding_entity']).agg({'id': 'count'}).unstack()
fei

In [ ]:
#asfe = age_state_fe.groupby(['demographic_distribution', 'funding_entity']).agg({'id': 'count'})

In [ ]:
#type(eval(new_df['region_distribution'][93568])), type(eval(new_df['region_distribution'][93568])[0]), eval(new_df['region_distribution'][93568])[0]

In [ ]:
#new_df = top_fe.loc[top_fe['pred_stance'] == '1'] #top10 pro-trump 38168 (not good)
#new_df = one_state_fe.loc[one_state_fe['pred_issue'] == 1] #covid

new_df = one_state_fe.loc[(one_state_fe['pred_stance'] == 1) | (one_state_fe['pred_stance'] == 2)] #pro-trump, anti-biden
#new_df = one_state_fe.loc[(one_state_fe['pred_stance'] == 0) | (one_state_fe['pred_stance'] == 3)] #pro-biden, anti-trump


In [ ]:
new_df.shape #17179
new_df['ad_creative_body'].unique()

In [ ]:
text = " ".join(review for review in new_df['ad_creative_body'].str.lower())
#text = " ".join(review.lower() for review in new_df['ad_creative_body'].unique())

#print ("There are {} words in the combination ads.".format(len(text)))

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#get stop words of all languages
#STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
# # Create stopword list:
# stopwords = set(STOPWORDS)
stop_words.update(["would","get","going","across","held","join","make","have","not"
                   "need","know","will","be","is","us","now","tommorow","today","yesterday","take",
                   "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
                  "want","three","please","ever","years","months","days","hour","hours","amount","wants",
                   "show","pm","am","need","let","use","used","really","think","new","party","name", "59pm"
                   ])

# stop_words.update(["would","get","going","across","held","join","joe", "biden", "kamala", "harris","make","donald","trump","pence","election","have","not"
#                    "need","know", "november","will","be","is","us","now","tommorow","today","yesterday","take","democrats","democrat","republican", "republicans",
#                    "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
#                   "want","three","please","ever","years","months","days","hour","hours","amount","wants","democratic",
#                    "show","pm","am","need","let","contribute","use","used","really","think","new","party","name",
#                    "defeat","vote","bernie","sanders","presidential","president","ballot","campaign","see","donation",
#                    "chip","candidate","moven","org", "senate","donate","one","people","jill","poll","niece","survey","minutes",
#                   "http","trump2020","coin","https","www","trump2020","biden-harris","trump-pence","biden2020","com","moveon",
#                    "conservatecollections",'hat','conservatecollection','conservativecollections.','even','give'])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stop_words, background_color="white", max_words=100).generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#wordcloud.to_file("image/PA_unbias_america_1st.png")
#wordcloud.to_file("image/PA_unbias_union2020.png")
#wordcloud.to_file("image/pres_trump.png")
#wordcloud.to_file("image/pres_biden.png")

## LDA Topic modeling

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = new_df.ad_creative_body.values.tolist()
data_words = list(sent_to_words(data))
#print(data_words[:1])
#data_words

In [ ]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
#from gensim.utils import lemmatize, simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 
#                    'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 
#                    'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 
#                    'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 
#                    'https','http','www','rt','am','pm','co','com','amp','&','nhttps','amit','late'])

nlp = spacy.load('en_core_web_sm')#python -m spacy download en


In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], min_count=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#print(bigram_mod)
# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    #print(text)
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    #print(texts)
    texts = [bigram_mod[doc] for doc in texts]
    #print(texts)
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    #print(texts)
    #sys.exit()
    texts_out = []
    
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    #texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics = 2, 
                                           random_state=100,
                                           #update_every=1,
                                           chunksize=2000,
                                           passes=20,
                                           alpha='auto',
                                           iterations=400,
                                           eval_every = None,
                                           per_word_topics=True)



top_topics = lda_model.top_topics(corpus, topn=10) #top 10 high frequency words of specific topic

from pprint import pprint
pprint(top_topics)

#pprint(lda_model.print_topics())
#> [(0,

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(num_words=10, formatted=False)

#fig, axes = plt.subplots(2, 3, figsize=(10,10), sharex=True, sharey=True)
fig, axes = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

#plt.savefig("image/apf_in_blue.png") #american potential fund in blue state topic
#plt.savefig("image/iu_in_red.png") #independence usa red topic
plt.show()

## Swing state info for Wisconsin, Pennsylvania

In [ ]:
# #Swing state info for Wisconsin, Pennsylvania
# ad_id = []
# stance_p = []
# text = []
# fe = []
# start = []
# stop = []
# imp = []
# spend = []
# region = []
# demo = []
# issue_p = []
# for j in range (0, predicted_fb_all.shape[0] ):
#     dic = eval(predicted_fb_all['region_distribution'][j])[0]
#     if dic['region'] == 'Wisconsin' or dic['region'] == 'Pennsylvania':
#         ad_id.append(predicted_fb_all.id[j])
#         text.append(predicted_fb_all.ad_creative_body[j])
#         fe.append(predicted_fb_all.funding_entity[j])
#         start.append(predicted_fb_all.ad_delivery_start_time[j])
#         stop.append(predicted_fb_all.ad_delivery_stop_time[j])
#         imp.append(predicted_fb_all.impressions[j])
#         spend.append(predicted_fb_all.spend[j])
#         region.append(predicted_fb_all.region_distribution[j])
#         demo.append(predicted_fb_all.demographic_distribution[j])
#         stance_p.append(predicted_fb_all.pred_stance[j])
#         issue_p.append(predicted_fb_all.pred_issue[j])



            

#Only pennsylvania

#Swing state info for Wisconsin, Pennsylvania, Florida, Michighan
ad_id = []
stance_p = []
text = []
fe = []
start = []
stop = []
imp = []
spend = []
region = []
demo = []
issue_p = []
impression = []
imp_upper = []
imp_lower = []
spend_upper = []
spend_lower = []
# for j in range (0, predicted_fb_all.shape[0] ):
#     dic = eval(predicted_fb_all['region_distribution'][j])[0]
#     if dic['region'] == 'Pennsylvania' : #'Pennsylvania':
#         ad_id.append(predicted_fb_all.id[j])
#         text.append(predicted_fb_all.ad_creative_body[j])
#         fe.append(predicted_fb_all.funding_entity[j])
#         start.append(predicted_fb_all.ad_delivery_start_time[j])
#         stop.append(predicted_fb_all.ad_delivery_stop_time[j])
#         imp.append(predicted_fb_all.impressions[j])
#         spend.append(predicted_fb_all.spend[j])
#         region.append(predicted_fb_all.region_distribution[j])
#         demo.append(predicted_fb_all.demographic_distribution[j])
#         stance_p.append(predicted_fb_all.pred_stance[j])
#         issue_p.append(predicted_fb_all.pred_issue[j])


for j in range (0, predicted_fb_all.shape[0] ):
    list_dic = eval(predicted_fb_all['region_distribution'][j])
#     list_imp_dic = eval(predicted_fb_all['impressions'][j])
#     list_spend_dic = eval(predicted_fb_all['spend'][j])
#     print (list_dic, list_imp_dic, list_imp_dic['upper_bound'], list_spend_dic, list_spend_dic['lower_bound'])
#     sys.exit()
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'Pennsylvania' : #'Pennsylvania':
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            start.append(predicted_fb_all.ad_delivery_start_time[j])
            stop.append(predicted_fb_all.ad_delivery_stop_time[j])
            imp.append(predicted_fb_all.impressions[j])
            spend.append(predicted_fb_all.spend[j])
            region.append(predicted_fb_all.region_distribution[j])
            demo.append(predicted_fb_all.demographic_distribution[j])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])

                                                           
                                                           
# for j in range (0, swing.shape[0] ):
#     list_dic = eval(swing['demographic_distribution'][j])
#     for i in range (0, len(list_dic)):
#         #print(list_dic[i])  
#         dic = list_dic[i]
#         #print(dic['age'] )
#         #sys.exit()
#         if dic['age'] == '13-17' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])

In [ ]:
#eval(predicted_fb_all['region_distribution'][0])[0]['region']

In [ ]:
swing = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'ad_delivery_start_time': start, 'ad_delivery_stop_time' : stop,
                             'impressions' : imp, 'spend' : spend, 'region_distribution': region,
                             'demographic_distribution': demo, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            })

In [ ]:
swing #451576 

In [ ]:
impression = []

for j in range (0, swing.shape[0] ):
    list_dic = eval(swing['region_distribution'][j])
    #print(list_dic)
    #sys.exit()
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'Pennsylvania' :
            impression.append(dic['percentage'])
#         print(dic)
#         sys.exit()
        
        

In [ ]:
len(impression)

In [ ]:
swing['reg_imp'] = impression
swing['reg_imp'] =  swing['reg_imp'].astype(float)
swing

In [ ]:
# sort_swing = swing.sort_values(by = 'reg_imp', ascending = False)
# print(sort_swing)
top_swing = swing.loc[(swing['reg_imp'] > 0.1)] #ads having > 10% regional impressions in PA
top_swing = top_swing.reset_index(drop=True)
print(top_swing)

In [ ]:
#drop "none" (-1) issue, state
new_swing = top_swing[top_swing.pred_issue != -1]
new_swing2 = new_swing[new_swing.pred_stance != -1]
new_swing2 = new_swing2.reset_index(drop=True)
new_swing2

In [ ]:
###Use groupby() function to form groups based on more than one category (i.e. Use more than one column to perform the splitting).
## gkk = df.groupby(['Team', 'Position']) 
###for color coded
dfc = new_swing2.groupby(['pred_issue', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
#dfc['id'][0].values


In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.6, color=['green', 'red', 'orange', 'blue']);
plt.title('Issue distribution with stances in PA', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Issue', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#ylabels = g.get_yticks().tolist()
#print(ylabels, type(ylabels[0]))
#ylabels[0] = 'abortion'

ylabels = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
    
g.set_yticklabels(ylabels)
g.legend(newnames,loc=(4), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/issue_PA.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
# swing.pred_stance.value_counts().plot.bar()
# plt.savefig('data/swing_stance_dis.png')

# swing.pred_issue.value_counts().plot.bar()
# plt.savefig('data/swing_issue_dis.png')



In [ ]:
#####swing states (PA, WI)  two different funding entities 
# state_fe = swing.loc[(swing['funding_entity'] == 'America First Action') | 
#                               (swing['funding_entity'] == 'Union 2020') ]


state_fe = swing.loc[(swing['funding_entity'] == 'BIDEN FOR PRESIDENT') | 
                              (swing['funding_entity'] == 'DONALD J. TRUMP FOR PRESIDENT, INC.') ]

## DONALD J. TRUMP FOR PRESIDENT, INC., #TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE
#state_fe = swing.loc[(swing['funding_entity'] == 'BIDEN FOR PRESIDENT') ]
#state_fe = swing.loc[(swing['funding_entity'] == 'Union 2020') ]
state_fe.reset_index(inplace=True) # Resets the index, makes factor a column

In [ ]:
state_fe

In [ ]:
sfe = state_fe.groupby(['pred_stance']).agg({'id': 'count'})
sfe

In [ ]:
#state_fe.to_csv('data/state_fe_PA_Union 2020.csv', line_terminator='\r\n', encoding='utf-8', index = False) 

In [ ]:
##### Demographic age for specific state for two funding entities
#   - 18-24
#   - 25-34
#   - 35-44
#   - 45-54
#   - 55-64
#   - 65+

ad_id = []
stance_p = []
text = []
fe = []
age = []
issue_p = []

for j in range (0, state_fe.shape[0] ):
    #print(state_fe['demographic_distribution'][j])
    list_dic = eval(state_fe['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        if dic['age'] == '13-17' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
        elif dic['age'] == '18-24' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
            
        elif dic['age'] == '25-34' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
        
        elif dic['age'] == '35-44' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
            
        elif dic['age'] == '45-54' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
        
        elif dic['age'] == '55-64' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
        
        elif dic['age'] == '65+' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])

In [ ]:
age_state_fe = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'demographic_distribution': age, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
age_state_fe

In [ ]:
asfe = age_state_fe.groupby(['demographic_distribution', 'funding_entity']).agg({'id': 'count'}).unstack()
#asfe = age_state_fe.groupby(['demographic_distribution']).agg({'id': 'count'})
asfe

In [ ]:
##### ##### Demographic gender for specific state for two funding entities
# demo_genders:
#   - male
#   - female
#   - unknown

ad_id = []
stance_p = []
text = []
fe = []
gender = []
issue_p = []

for j in range (0, state_fe.shape[0] ):
    list_dic = eval(state_fe['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        
        if dic['gender'] == 'male' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
            
        elif dic['gender'] == 'female' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
            
        elif dic['gender'] == 'unknown' :
            ad_id.append(state_fe.id[j])
            text.append(state_fe.ad_creative_body[j])
            fe.append(state_fe.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(state_fe.pred_stance[j])
            issue_p.append(state_fe.pred_issue[j])
        

In [ ]:
gender_state_fe = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'demographic_distribution': gender, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
gsfe = gender_state_fe.groupby(['demographic_distribution', 'funding_entity']).agg({'id': 'count'}).unstack()
#gsfe = gender_state_fe.groupby(['demographic_distribution']).agg({'id': 'count'})
gsfe

## Blue state NY

In [ ]:
# #Blue state info for California , New York
# ad_id = []
# stance_p = []
# text = []
# fe = []
# start = []
# stop = []
# imp = []
# spend = []
# region = []
# demo = []
# issue_p = []
# for j in range (0, predicted_fb_all.shape[0] ):
#     dic = eval(predicted_fb_all['region_distribution'][j])[0]
#     if dic['region'] == 'New York' or dic['region'] == 'California':
#         ad_id.append(predicted_fb_all.id[j])
#         text.append(predicted_fb_all.ad_creative_body[j])
#         fe.append(predicted_fb_all.funding_entity[j])
#         start.append(predicted_fb_all.ad_delivery_start_time[j])
#         stop.append(predicted_fb_all.ad_delivery_stop_time[j])
#         imp.append(predicted_fb_all.impressions[j])
#         spend.append(predicted_fb_all.spend[j])
#         region.append(predicted_fb_all.region_distribution[j])
#         demo.append(predicted_fb_all.demographic_distribution[j])
#         stance_p.append(predicted_fb_all.pred_stance[j])
#         issue_p.append(predicted_fb_all.pred_issue[j])

#Blue state info for New York
ad_id = []
stance_p = []
text = []
fe = []
start = []
stop = []
imp = []
spend = []
region = []
demo = []
issue_p = []
for j in range (0, predicted_fb_all.shape[0] ):
    list_dic = eval(predicted_fb_all['region_distribution'][j])
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'New York' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            start.append(predicted_fb_all.ad_delivery_start_time[j])
            stop.append(predicted_fb_all.ad_delivery_stop_time[j])
            imp.append(predicted_fb_all.impressions[j])
            spend.append(predicted_fb_all.spend[j])
            region.append(predicted_fb_all.region_distribution[j])
            demo.append(predicted_fb_all.demographic_distribution[j])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        


                                                      

In [ ]:
blue = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'ad_delivery_start_time': start, 'ad_delivery_stop_time' : stop,
                             'impressions' : imp, 'spend' : spend, 'region_distribution': region,
                             'demographic_distribution': demo, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) #34200, (NY = 5552)

In [ ]:
blue

In [ ]:
impression = []

for j in range (0, blue.shape[0] ):
    list_dic = eval(blue['region_distribution'][j])
    #print(list_dic)
    #sys.exit()
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'New York' :
            impression.append(dic['percentage'])
#         print(dic)
#         sys.exit()
        

In [ ]:
blue['reg_imp'] = impression
blue['reg_imp'] =  blue['reg_imp'].astype(float)
blue

In [ ]:
top_blue = blue.loc[(blue['reg_imp'] > 0.1)] #ads having > 10% regional impressions in NY
top_blue = top_blue.reset_index(drop=True)
print(top_blue)

In [ ]:
#drop "none" (-1) issue, stance
#drop "none" (-1) issue, state

new_blue = top_blue[top_blue.pred_issue != -1]
new_blue2 = new_blue[new_blue.pred_stance != -1]
new_blue2 = new_blue2.reset_index(drop=True)
new_blue2

In [ ]:
###Use groupby() function to form groups based on more than one category (i.e. Use more than one column to perform the splitting).
dfc = new_blue2.groupby(['pred_issue', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.6, color=['green', 'red', 'orange', 'blue']);
plt.title('Issue distribution with stances in NY', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Issue', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#ylabels = g.get_yticks().tolist()
#print(ylabels, type(ylabels[0]))
#ylabels[0] = 'abortion'

ylabels = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
    
g.set_yticklabels(ylabels)
g.legend(newnames,loc=(4), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/issue_NY.png", dpi=300, bbox_inches = "tight") 
plt.show()

## Red state Idaho

In [ ]:

#Red state info for Idaho 
ad_id = []
stance_p = []
text = []
fe = []
start = []
stop = []
imp = []
spend = []
region = []
demo = []
issue_p = []
for j in range (0, predicted_fb_all.shape[0] ):
    list_dic = eval(predicted_fb_all['region_distribution'][j])
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'Idaho' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            start.append(predicted_fb_all.ad_delivery_start_time[j])
            stop.append(predicted_fb_all.ad_delivery_stop_time[j])
            imp.append(predicted_fb_all.impressions[j])
            spend.append(predicted_fb_all.spend[j])
            region.append(predicted_fb_all.region_distribution[j])
            demo.append(predicted_fb_all.demographic_distribution[j])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        

In [ ]:
red = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'ad_delivery_start_time': start, 'ad_delivery_stop_time' : stop,
                             'impressions' : imp, 'spend' : spend, 'region_distribution': region,
                             'demographic_distribution': demo, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) #8659 ##idaho = 4346

In [ ]:
red 

In [ ]:
impression = []

for j in range (0, red.shape[0] ):
    list_dic = eval(red['region_distribution'][j])
    #print(list_dic)
    #sys.exit()
    for i in range (0, len(list_dic)):
        dic = list_dic[i]
        if dic['region'] == 'Idaho' :
            impression.append(dic['percentage'])
#         print(dic)
#         sys.exit()
        

In [ ]:
red['reg_imp'] = impression
red['reg_imp'] =  red['reg_imp'].astype(float)
red

In [ ]:
top_red = red.loc[(red['reg_imp'] > 0.1)] #ads having > 10% regional impressions in ID
top_red = top_red.reset_index(drop=True)
print(top_red)

In [ ]:
#drop "none" (-1) issue
new_red = top_red[top_red.pred_issue != -1]
new_red2 = new_red[new_red.pred_stance != -1]
new_red2 = new_red2.reset_index(drop=True)
new_red2

In [ ]:
dfc = new_red2.groupby(['pred_issue', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.6, color=['green', 'red', 'orange', 'blue']);
plt.title('Issue distribution with stances in ID', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Issue', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#ylabels = g.get_yticks().tolist()
#print(ylabels, type(ylabels[0]))
#ylabels[0] = 'abortion'

ylabels = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
    
g.set_yticklabels(ylabels)
g.legend(newnames,loc=(4), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/issue_ID.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
###issue of 6 states
#new_df = swing.loc[predicted_fb_all['pred_issue'] == '4'] #economy
#new_df = swing.loc[predicted_fb_all['pred_issue'] == '10'] #supreme court

In [ ]:
#df["first_column"]. str. lower()
text = " ".join(review for review in new_df['ad_creative_body'].str.lower())
print ("There are {} words in the combination of all pro-biden ads.".format(len(text)))

In [ ]:
#get stop words of all languages
#STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
# # Create stopword list:
# stopwords = set(STOPWORDS)
# stop_words.update(["would","get","going","across","held","join","joe", "biden", "kamala", "harris","make","donald","trump","pence","election","have","not"
#                    "need","know", "november","will","be","is","us","now","tommorow","today","yesterday","take","democrats","democrat","republican", "republicans",
#                    "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
#                   "want","three","please","ever","years","months","days","hour","hours","amount","wants","democratic",
#                    "show","pm","am","need","let","contribute","use","used","really","think","new","party","name",
#                    ])

stop_words.update(["would","get","going","across","held","join","joe", "biden", "kamala", "harris","make","donald","trump","pence","election","have","not"
                   "need","know", "november","will","be","is","us","now","tommorow","today","yesterday","take","democrats","democrat","republican", "republicans",
                   "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
                  "want","three","please","ever","years","months","days","hour","hours","amount","wants","democratic",
                   "show","pm","am","need","let","contribute","use","used","really","think","new","party","name",
                   "defeat","vote","bernie","sanders","presidential","president","ballot","campaign","see","donation",
                   "chip","candidate","moven","org", "senate","donate","one","people","jill","poll","niece","survey","minutes",
                  "http","trump2020","coin","https","www","trump2020","biden-harris","trump-pence","biden2020","com","moveon",
                   "conservatecollections",'hat','conservatecollection','conservativecollections.','even','give','long','trying'])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stop_words, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#wordcloud.to_file("data/swing_economy_wc.png")

## Descriptive insights of data 

In [ ]:
###Use groupby() function to form groups based on more than one category (i.e. Use more than one column to perform the splitting).
#issue distribution over stance
#dfc = predicted_fb_all.groupby(['pred_issue', 'pred_stance']).agg({'id': 'count'})
#stance distribution over issue
dfc = predicted_fb_all.groupby(['pred_stance', 'pred_issue']).agg({'id': 'count'})
dfc

## wordcloud for stances

In [ ]:
#new_df = predicted_fb_all.loc[(predicted_fb_all['pred_stance'] == 0)] #pro-biden
#new_df = predicted_fb_all.loc[(predicted_fb_all['pred_stance'] == 1)] #pro-trump
new_df = predicted_fb_all.loc[(predicted_fb_all['pred_stance'] == 2)] #anti-biden
#new_df = predicted_fb_all.loc[(predicted_fb_all['pred_stance'] == 3)] #anti-trump



In [ ]:
text = " ".join(review for review in new_df['ad_creative_body'].str.lower())

print ("There are {} words in the combination ads.".format(len(text)))

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#get stop words of all languages
#STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
# # Create stopword list:
# stopwords = set(STOPWORDS)
stop_words.update(["would","get","going","across","held","join","joe", "biden", "kamala", "harris","make","donald","trump","pence","election","have","not"
                   "need","know", "november","will","be","is","us","now","tommorow","today","yesterday","take","democrats","democrat","republican", "republicans",
                   "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
                  "want","three","please","ever","years","months","days","hour","hours","amount","wants","democratic",
                   "show","pm","am","need","let","contribute","use","used","really","think","new","party","name",
                   ])

# stop_words.update(["would","get","going","across","held","join","joe", "biden", "kamala", "harris","make","donald","trump","pence","election","have","not"
#                    "need","know", "november","will","be","is","us","now","tommorow","today","yesterday","take","democrats","democrat","republican", "republicans",
#                    "sure", "country","state", "county","p","tonight","like","ready","time","tell","midnight","year","month","week","day",
#                   "want","three","please","ever","years","months","days","hour","hours","amount","wants","democratic",
#                    "show","pm","am","need","let","contribute","use","used","really","think","new","party","name",
#                    "defeat","vote","bernie","sanders","presidential","president","ballot","campaign","see","donation",
#                    "chip","candidate","moven","org", "senate","donate","one","people","jill","poll","niece","survey","minutes",
#                   "http","trump2020","coin","https","www","trump2020","biden-harris","trump-pence","biden2020","com","moveon",
#                    "conservatecollections",'hat','conservatecollection','conservativecollections.','even','give'])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stop_words, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Demographic analysis for whole data with stance and issue

In [ ]:
avg_imp = []
avg_spend = []
for j in range (0, predicted_fb_all.shape[0] ):
    dic_imp = eval(predicted_fb_all['impressions'][j])
    dic_cost = eval(predicted_fb_all['spend'][j])
    dic_imp_new = dict([k, float(v)] for k, v in dic_imp.items()) 
    dic_cost_new = dict([k, float(v)] for k, v in dic_cost.items()) 
    #print(dic_imp_new) 
    imp = sum(dic_imp_new.values()) / len(dic_imp_new)
    avg_imp.append(imp) # avg of impression
    cost = sum(dic_cost_new.values()) / len(dic_cost_new)
    avg_spend.append(cost) # avg of spend

In [ ]:
predicted_fb_all['avg_imp'] = avg_imp
predicted_fb_all['avg_spend'] = avg_spend
predicted_fb_all['date']= pd.to_datetime(predicted_fb_all['ad_delivery_start_time']) #convert string to date-time

In [ ]:
#eval(predicted_fb_all['demographic_distribution'][100]) #list

In [ ]:
#eval(predicted_fb_all['demographic_distribution'][100])[0]

In [ ]:
##### Demographic age
# - 13-17
#   - 18-24
#   - 25-34
#   - 35-44
#   - 45-54
#   - 55-64
#   - 65+

ad_id = []
stance_p = []
text = []
fe = []
age = []
issue_p = []
imp = []
for j in range (0, predicted_fb_all.shape[0] ):
    list_dic = eval(predicted_fb_all['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        if dic['age'] == '13-17' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
            
        elif dic['age'] == '18-24' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
            
        elif dic['age'] == '25-34' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        
        elif dic['age'] == '35-44' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
            
        elif dic['age'] == '45-54' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        
        elif dic['age'] == '55-64' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        
        elif dic['age'] == '65+' :
            ad_id.append(predicted_fb_all.id[j])
            text.append(predicted_fb_all.ad_creative_body[j])
            fe.append(predicted_fb_all.funding_entity[j])
            imp.append(predicted_fb_all.avg_imp[j])
            age.append(dic['age'])
            stance_p.append(predicted_fb_all.pred_stance[j])
            issue_p.append(predicted_fb_all.pred_issue[j])
        

In [ ]:
age_df = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'age': age, 'avg_imp' : imp, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
age_df

In [ ]:
# dfc = age_df.groupby(['age', 'pred_stance']).agg({'id': 'count'})
dfc = age_df.groupby(['pred_stance', 'age']).agg({'id': 'count'})
dfc

In [ ]:
# gender_age_summary=age_df.groupby(['pred_stance','age']).agg(tot_stance=('id','count')).unstack()
# gender_age_summary.head()
# gender_age_summary.plot(kind='bar', figsize=(10,6), stacked=True);
# plt.title('Summary of stances by Gender & Age');
# plt.xlabel('Age');
# plt.ylabel('#Stances');

In [ ]:
##### Demographic gender
# demo_genders:
#   - male
#   - female
#   - unknown

# ad_id = []
# stance_p = []
# text = []
# fe = []
gender = []
# issue_p = []
impression = []
for j in range (0, predicted_fb_all.shape[0] ):
    list_dic = eval(predicted_fb_all['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        impression.append(dic['percentage'])
        if dic['gender'] == 'male' :
#             ad_id.append(predicted_fb_all.id[j])
#             text.append(predicted_fb_all.ad_creative_body[j])
#             fe.append(predicted_fb_all.funding_entity[j])
            gender.append(dic['gender'])

#             stance_p.append(predicted_fb_all.pred_stance[j])
#             issue_p.append(predicted_fb_all.pred_issue[j])
            
        elif dic['gender'] == 'female' :
#             ad_id.append(predicted_fb_all.id[j])
#             text.append(predicted_fb_all.ad_creative_body[j])
#             fe.append(predicted_fb_all.funding_entity[j])
            gender.append(dic['gender'])
#             stance_p.append(predicted_fb_all.pred_stance[j])
#             issue_p.append(predicted_fb_all.pred_issue[j])
        elif dic['gender'] == 'unknown' :
#             ad_id.append(predicted_fb_all.id[j])
#             text.append(predicted_fb_all.ad_creative_body[j])
#             fe.append(predicted_fb_all.funding_entity[j])
            gender.append(dic['gender'])
#             stance_p.append(predicted_fb_all.pred_stance[j])
#             issue_p.append(predicted_fb_all.pred_issue[j])
        

In [ ]:
gender_df = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'gender': gender, 'ag_imp':impression, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
age_df['gender'] = gender
age_df['ag_imp'] = impression

In [ ]:
age_df

## stance vs demographic

In [ ]:
#drop "none" (-1) stance
new_age_df = age_df[age_df.pred_stance != -1]

In [ ]:
new_age_df.id.nunique() #852790

In [ ]:
gender_age_summary=new_age_df.groupby(['age', 'gender','pred_stance']).agg(stance=('id','count')).unstack()
gender_age_summary.head()
g = gender_age_summary.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['green', 'red', 'orange', 'blue']);
plt.title('Summary of stances by age & gender', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Stances', fontsize=12);
plt.xticks(rotation=90)
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/11.76]
g.set_yticklabels(ylabels)
plt.savefig("image/age_gender_stance.png", dpi=300, bbox_inches = "tight") 
plt.show()


In [ ]:
gender_age_summary

## issue vs demographic

In [ ]:
#drop "none" (-1) issue
new_age_df = age_df[age_df.pred_issue != -1]
new_age_df = new_age_df.reset_index(drop=True)
new_age_df

In [ ]:
gender_age_summary=new_age_df.groupby(['age', 'gender','pred_issue']).agg(stance=('id','count')).unstack()
gender_age_summary.head()
g = gender_age_summary.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['coral', 'green', 'pink', 'blue', '#EDB120', 'cyan', 'brown', 'black', 'lime', 'magenta', 'red', 'gray','purple']);
plt.title('Summary of issues by age & gender', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Issues', fontsize=12);
plt.xticks(rotation=90)
newnames = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/11.76]
g.set_yticklabels(ylabels)
plt.savefig("image/age_gender_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
#gender_age_summary.to_csv("data/aba.csv")

In [ ]:
# dfc = gender_df.groupby(['gender', 'pred_stance']).agg({'id': 'count'})
# dfc = gender_df.groupby(['pred_stance', 'gender']).agg({'id': 'count'})
# dfc

## impression vs demographic


### check pro-biden, pro-trump, anti-biden, anti-trump stance separately

In [ ]:
#stance_imp_df = age_df.loc[(age_df['pred_stance'] == 0) | (age_df['pred_stance'] == 1) ]
stance_imp_df = age_df.loc[(age_df['pred_stance'] == 0)] #pro-biden , 341335
#stance_imp_df = age_df.loc[(age_df['pred_stance'] == 1)] #pro-trump , 5517397
#stance_imp_df = age_df.loc[(age_df['pred_stance'] == 2)] #anti-biden , 1341884
#stance_imp_df = age_df.loc[(age_df['pred_stance'] == 3)] #anti-trump, 2827715, 10028331
stance_imp_df = stance_imp_df.reset_index(drop=True)
#convert value of specific column of dataframe str to flot 
stance_imp_df['ag_imp'] =  stance_imp_df['ag_imp'].astype(float)
#stance_imp_df["ag_imp"] = 100 * stance_imp_df["ag_imp"]
stance_imp_df["ag_imp"] = (stance_imp_df["avg_imp"] * stance_imp_df["ag_imp"])
stance_imp_df

In [ ]:

# #group by age, gender based on sum of impression of age gender in pro-biden, anti trump ad
imp = stance_imp_df.groupby([ 'age' , 'gender']).agg({'ag_imp': 'sum'}).unstack()
print(imp)


In [ ]:
stance_imp_df.ag_imp.sum()/1000000 # total impression = 11733.104273 M (11.7 B)
# pb = 669 M, pt = 5241 M, ab = 1585, at = 5099

In [ ]:
#imp.plot.barh()
g = imp.plot(kind='barh', figsize=(8,5), width = 0.5, color=['blue', 'lime', 'red']);
plt.title('Summary of impressions by age & gender', fontsize=14);
plt.xlabel('Impressions', fontsize=12);
plt.ylabel('Age group', fontsize=12);
xlabels = ['{}'.format(x) + 'M' for x in g.get_xticks()/1000000]
newnames = ['male', 'female', 'unknown']
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(4), fontsize=12)
#plt.savefig("image/age_gender_impression_pro_biden.png", dpi=300, bbox_inches = "tight") 
#plt.savefig("image/age_gender_impression_pro_trump.png", dpi=300, bbox_inches = "tight") 
#plt.savefig("image/age_gender_impression_anti_biden.png", dpi=300, bbox_inches = "tight") 
#plt.savefig("image/age_gender_impression_anti_trump.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
# # ### which funding entity has high impression from demographics
# imp = age_df.groupby(['funding_entity'], as_index=False).agg({'ag_imp': 'mean'})
# #imp = pb_df.groupby(['funding_entity '], as_index=False).agg({'ag_imp': 'sum'})
# imp = age_df.groupby(['funding_entity', 'age', 'gender'], as_index=False).agg({'ag_imp': 'mean'})
# print(imp.sort_values(by = 'ag_imp', ascending = False))
# print(imp)

## State-wise Issue and Demographics

### Swing state - PA, age distribution

In [ ]:
# ### age PA

# ##### Demographic age
# #   - 18-24
# #   - 25-34
# #   - 35-44
# #   - 45-54
# #   - 55-64
# #   - 65+

# ad_id = []
# stance_p = []
# text = []
# fe = []
# age = []
# issue_p = []

# for j in range (0, swing.shape[0] ):
#     list_dic = eval(swing['demographic_distribution'][j])
#     for i in range (0, len(list_dic)):
#         #print(list_dic[i])  
#         dic = list_dic[i]
#         #print(dic['age'] )
#         #sys.exit()
#         if dic['age'] == '13-17' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
#         elif dic['age'] == '18-24' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
#         elif dic['age'] == '25-34' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
        
#         elif dic['age'] == '35-44' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
            
#         elif dic['age'] == '45-54' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
        
#         elif dic['age'] == '55-64' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
        
#         elif dic['age'] == '65+' :
#             ad_id.append(swing.id[j])
#             text.append(swing.ad_creative_body[j])
#             fe.append(swing.funding_entity[j])
#             age.append(dic['age'])
#             stance_p.append(swing.pred_stance[j])
#             issue_p.append(swing.pred_issue[j])
            
        

In [ ]:

### age PA

##### Demographic age
#   - 18-24
#   - 25-34
#   - 35-44
#   - 45-54
#   - 55-64
#   - 65+

ad_id = []
stance_p = []
text = []
fe = []
age = []
issue_p = []

for j in range (0, top_swing.shape[0] ):
    list_dic = eval(top_swing['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        if dic['age'] == '13-17' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        elif dic['age'] == '18-24' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        elif dic['age'] == '25-34' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
        
        elif dic['age'] == '35-44' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        elif dic['age'] == '45-54' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
        
        elif dic['age'] == '55-64' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        
        elif dic['age'] == '65+' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        

In [ ]:
age_swing = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'age_distribution': age, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_age_swing = age_swing[age_swing.pred_stance != -1]
new_age_swing = new_age_swing.reset_index(drop=True)
new_age_swing

In [ ]:
dfc = new_age_swing.groupby(['age_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
new_age_swing.id.nunique() #102439

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Age distribution with stances in PA', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Age Group', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/11.5]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(4), fontsize=9)
#g.get_legend().remove()
plt.savefig("image/age_PA.png", dpi=300, bbox_inches = "tight") 
plt.show()

### Blue state - NY, age distribution

In [ ]:
### age NY

##### Demographic age
#   - 18-24
#   - 25-34
#   - 35-44
#   - 45-54
#   - 55-64
#   - 65+

ad_id = []
stance_p = []
text = []
fe = []
age = []
issue_p = []

for j in range (0, top_blue.shape[0] ):
    list_dic = eval(top_blue['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        if dic['age'] == '13-17' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        elif dic['age'] == '18-24' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        elif dic['age'] == '25-34' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        
        elif dic['age'] == '35-44' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
            
        elif dic['age'] == '45-54' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        
        elif dic['age'] == '55-64' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        
        elif dic['age'] == '65+' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        

In [ ]:
age_blue = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'age_distribution': age, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_age_blue = age_blue[age_blue.pred_stance != -1]
new_age_blue = new_age_blue.reset_index(drop=True)
new_age_blue

In [ ]:
dfc = new_age_blue.groupby(['age_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
new_age_blue.id.nunique() #44586

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Age distribution with stances in NY', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Age Group', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
# xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/10.9]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(4), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/age_NY.png", dpi=300, bbox_inches = "tight") 
plt.show()

### Red state - ID, age distribution

In [ ]:
### age Idaho

##### Demographic age
# - 13-17
#   - 18-24
#   - 25-34
#   - 35-44
#   - 45-54
#   - 55-64
#   - 65+

ad_id = []
stance_p = []
text = []
fe = []
age = []
issue_p = []

for j in range (0, top_red.shape[0] ):
    list_dic = eval(top_red['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        
        if dic['age'] == '13-17' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        elif dic['age'] == '18-24' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
            
        elif dic['age'] == '25-34' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        
        elif dic['age'] == '35-44' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
            
        elif dic['age'] == '45-54' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        
        elif dic['age'] == '55-64' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        
        elif dic['age'] == '65+' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            age.append(dic['age'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        

In [ ]:
age_red = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'age_distribution': age, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_age_red = age_red[age_red.pred_stance != -1]
new_age_red = new_age_red.reset_index(drop=True)
new_age_red

In [ ]:
dfc = new_age_red.groupby(['age_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

# # #group by age, gender based on sum of impression of age gender in pro-biden, anti trump ad
# imp = stance_imp_df.groupby([ 'age' , 'gender']).agg({'ag_imp': 'sum'}).unstack()
# print(imp)

In [ ]:
new_age_red.id.nunique() #1406

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Age distribution with stances in ID', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Age Group', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/4.9]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(4), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/age_ID.png", dpi=300, bbox_inches = "tight") 
plt.show()

### Swing state - PA, gender distribution

In [ ]:
##### Demographic gender #PA
# demo_genders:
#   - male
#   - female
#   - unknown

ad_id = []
stance_p = []
text = []
fe = []
gender = []
issue_p = []

for j in range (0, top_swing.shape[0] ):
    list_dic = eval(top_swing['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        
        if dic['gender'] == 'male' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        elif dic['gender'] == 'female' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
            
        elif dic['gender'] == 'unknown' :
            ad_id.append(top_swing.id[j])
            text.append(top_swing.ad_creative_body[j])
            fe.append(top_swing.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_swing.pred_stance[j])
            issue_p.append(top_swing.pred_issue[j])
        

In [ ]:
swing_gender = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'gender_distribution': gender, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_swing_gender = swing_gender[swing_gender.pred_stance != -1]
new_swing_gender = new_swing_gender.reset_index(drop=True)
new_swing_gender

In [ ]:
dfc = new_swing_gender.groupby(['gender_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
new_swing_gender.id.nunique() #102439

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Gender distribution with stances in PA', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Gender', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/11.5]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(1), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/gen_PA.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
#age_swing  swing_gender

In [ ]:
age_swing['gender'] = gender


In [ ]:
age_swing

In [ ]:
st1 = age_swing[age_swing.pred_stance != -1]
st2 = st1[st1.pred_issue != -1]
st2 = st2.reset_index(drop=True)
st2

In [ ]:
st2.id.nunique() #424895

In [ ]:
st=st2.groupby(['age_distribution', 'gender','pred_stance']).agg(stance=('id','count')).unstack()
st.head()
g = st.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['green', 'red', 'orange', 'blue']);
plt.title('Summary of stances by age & gender in PA', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Stances', fontsize=12);
plt.xticks(rotation=90)
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
#plt.savefig("image/age_gender_stance.png", dpi=300, bbox_inches = "tight") 
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/14.4]
g.set_yticklabels(ylabels)
plt.savefig("image/PA_age_gender_stance.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
st=st2.groupby(['age_distribution', 'gender','pred_issue']).agg(stance=('id','count')).unstack()
st.head()
g = st.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['coral', 'green', 'pink', 'blue', '#EDB120', 'cyan', 'brown', 'black', 'lime', 'magenta', 'red', 'gray','purple']);
plt.title('Summary of issues by age & gender in PA', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Issues', fontsize=12);
plt.xticks(rotation=90)
newnames = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/14.4]
g.set_yticklabels(ylabels)
plt.savefig("image/cor_PA_age_gender_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()

### Blue state - NY, gender distribution

In [ ]:
##### Demographic gender #NY
# demo_genders:
#   - male
#   - female
#   - unknown

ad_id = []
stance_p = []
text = []
fe = []
gender = []
issue_p = []

for j in range (0, top_blue.shape[0] ):
    list_dic = eval(top_blue['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        
        if dic['gender'] == 'male' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        elif dic['gender'] == 'female' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
            
        elif dic['gender'] == 'unknown' :
            ad_id.append(top_blue.id[j])
            text.append(top_blue.ad_creative_body[j])
            fe.append(top_blue.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_blue.pred_stance[j])
            issue_p.append(top_blue.pred_issue[j])
        

In [ ]:
blue_gender = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'gender_distribution': gender, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_blue_gender = blue_gender[blue_gender.pred_stance != -1]
new_blue_gender = new_blue_gender.reset_index(drop=True)
new_blue_gender

In [ ]:
dfc = new_blue_gender.groupby(['gender_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
new_blue_gender.id.nunique() #44586

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Gender distribution with stances in NY', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Gender', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/10.9]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(1), fontsize=10)
plt.savefig("image/gen_NY.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
age_blue['gender'] = gender
age_blue

In [ ]:
bt1 = age_blue[age_blue.pred_stance != -1]
bt2 = bt1[bt1.pred_issue != -1]
bt2 = bt2.reset_index(drop=True)
bt2

In [ ]:
bt2.id.nunique() #363737

In [ ]:
bt=bt2.groupby(['age_distribution', 'gender','pred_stance']).agg(stance=('id','count')).unstack()
bt.head()
g = bt.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['green', 'red', 'orange', 'blue']);
plt.title('Summary of stances by age & gender in NY', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Stances', fontsize=12);
plt.xticks(rotation=90)
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
#plt.savefig("image/age_gender_stance.png", dpi=300, bbox_inches = "tight") 
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/13.88]

g.set_yticklabels(ylabels)
plt.savefig("image/cor_NY_age_gender.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
bt=bt2.groupby(['age_distribution', 'gender','pred_issue']).agg(stance=('id','count')).unstack()
bt.head()
g = bt.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['coral', 'green', 'pink', 'blue', '#EDB120', 'cyan', 'brown', 'black', 'lime', 'magenta', 'red', 'gray','purple']);
plt.title('Summary of issues by age & gender in NY', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Issues', fontsize=12);
plt.xticks(rotation=90)
newnames = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/13.88]
g.set_yticklabels(ylabels)
plt.savefig("image/cor_NY_age_gender_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()

### Red state - ID, gender distribution

In [ ]:
##### Demographic gender #ID
# demo_genders:
#   - male
#   - female
#   - unknown

ad_id = []
stance_p = []
text = []
fe = []
gender = []
issue_p = []

for j in range (0, top_red.shape[0] ):
    list_dic = eval(top_red['demographic_distribution'][j])
    for i in range (0, len(list_dic)):
        #print(list_dic[i])  
        dic = list_dic[i]
        #print(dic['age'] )
        #sys.exit()
        
        if dic['gender'] == 'male' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
            
        elif dic['gender'] == 'female' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        elif dic['gender'] == 'unknown' :
            ad_id.append(top_red.id[j])
            text.append(top_red.ad_creative_body[j])
            fe.append(top_red.funding_entity[j])
            gender.append(dic['gender'])
            stance_p.append(top_red.pred_stance[j])
            issue_p.append(top_red.pred_issue[j])
        

In [ ]:
red_gender = pd.DataFrame({'id': ad_id, 'ad_creative_body': text, 'funding_entity': fe, 
                             'gender_distribution': gender, 'pred_stance' : stance_p, 'pred_issue' : issue_p
                            }) 

In [ ]:
new_red_gender = red_gender[red_gender.pred_stance != -1]
new_red_gender = new_red_gender.reset_index(drop=True)
new_red_gender

In [ ]:
dfc = new_red_gender.groupby(['gender_distribution', 'pred_stance']).agg({'id': 'count'}).unstack()
dfc

In [ ]:
new_red_gender.id.nunique() #1406

In [ ]:
g = dfc.plot(kind='barh', figsize=(8,5), width = 0.5, color=['green', 'red', 'orange', 'blue']);
plt.title('Gender distribution with stances in ID', fontsize=14);
plt.xlabel('#Stances', fontsize=12);
plt.ylabel('Gender', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
#xlabels = ['{}'.format(x) + 'K' for x in g.get_xticks()/1000]
xlabels = ['{}'.format(int(x)) for x in g.get_xticks()/4.9]
g.set_xticklabels(xlabels)
g.legend(newnames,loc=(1), fontsize=10)
#g.get_legend().remove()
plt.savefig("image/gen_ID.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
age_red['gender'] = gender
age_red

In [ ]:
rt1 = age_red[age_red.pred_stance != -1]
rt2 = rt1[rt1.pred_issue != -1]
rt2 = rt2.reset_index(drop=True)
rt2

In [ ]:
rt2.id.nunique() #286649

In [ ]:
rt=rt2.groupby(['age_distribution', 'gender','pred_stance']).agg(stance=('id','count')).unstack()
rt.head()
g = rt.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['green', 'red', 'orange', 'blue']);
plt.title('Summary of stances by age & gender in ID', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Stances', fontsize=12);
plt.xticks(rotation=90)
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
print(g.get_yticks(), sum (g.get_yticks()), g.get_yticks()/14.92, sum (g.get_yticks()/15))
#plt.savefig("image/age_gender_stance.png", dpi=300, bbox_inches = "tight") 
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/15]

g.set_yticklabels(ylabels)
plt.savefig("image/cor_ID_age_gender.png", dpi=300, bbox_inches = "tight") 
plt.show()

In [ ]:
rt=rt2.groupby(['age_distribution', 'gender','pred_issue']).agg(stance=('id','count')).unstack()
rt.head()
g = rt.plot(kind='bar', figsize=(10,6), width = 0.5, stacked=True, color=['coral', 'green', 'pink', 'blue', '#EDB120', 'cyan', 'brown', 'black', 'lime', 'magenta', 'red', 'gray','purple']);
plt.title('Summary of issues by age & gender in ID', fontsize=14);
plt.xlabel('Age Group, Gender', fontsize=12);
plt.ylabel('#Issues', fontsize=12);
plt.xticks(rotation=90)
newnames = ['abortion', 'covid', 'climate', 'law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
plt.legend(newnames,loc=(1.01,0), fontsize=12)
ylabels = ['{}'.format(int(y)) for y in g.get_yticks()/15]
g.set_yticklabels(ylabels)
plt.savefig("image/cor_ID_age_gender_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()

## Temporal Analysis

In [ ]:
new_pred = predicted_fb_all[predicted_fb_all.pred_issue != -1]
temp = new_pred[new_pred.pred_stance != -1]
temp = temp.reset_index(drop = True)
#temp

In [ ]:
hi = temp[temp.pred_stance != -1]
fi = hi[hi.pred_issue != -1]
fi.avg_imp.sum()/1000000 


In [ ]:
avg_imp = []
avg_spend = []
for j in range (0, temp.shape[0] ):
    dic_imp = eval(temp['impressions'][j])
    dic_cost = eval(temp['spend'][j])
    dic_imp_new = dict([k, float(v)] for k, v in dic_imp.items()) 
    dic_cost_new = dict([k, float(v)] for k, v in dic_cost.items()) 
    #print(dic_imp_new) 
    imp = sum(dic_imp_new.values()) / len(dic_imp_new)
    avg_imp.append(imp) # avg of impression
    cost = sum(dic_cost_new.values()) / len(dic_cost_new)
    avg_spend.append(cost) # avg of spend

In [ ]:
temp['avg_imp'] = avg_imp
temp['avg_spend'] = avg_spend
temp['date']= pd.to_datetime(temp['ad_delivery_start_time']) #convert string to date-time

In [ ]:
# #new_temp = temp.loc[(temp['pred_issue'] == 0 )]
# new_temp = temp.loc[(temp['pred_issue'] == 0) | 
#                       (temp['pred_issue'] == 3)]


In [ ]:
# new_temp

In [ ]:

# grp_temp = temp.groupby([ 'pred_stance' , 'date']).agg({'avg_imp': 'sum'})
# print(grp_temp)

In [ ]:

# pb = temp.resample('W-MON', on='date').avg_imp.sum()
# pb

## Timeseries of impressions based on 4 stances

In [ ]:
#Group by on stance and then resample 'weekly' on date and sum up average impression

# grp_temp = temp.groupby('pred_stance').resample('W-MON', on='date').agg({'avg_imp': 'sum'})
# print(grp_temp, grp_temp.shape) #169
# grp_temp.to_csv("data/bl.csv")

grp_temp = temp.pivot_table('avg_imp', index='date', columns='pred_stance', aggfunc='sum')
grp_temp

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.transforms as transforms
sns.set()  # use Seaborn styles
g = grp_temp.plot(kind='line', figsize=(15,6), color=['green', 'red', 'orange', 'blue'])
plt.xlabel('Time', fontsize=12);
plt.ylabel('Impressions', fontsize=12);
newnames = ['pro-biden', 'pro-trump', 'anti-biden', 'anti-trump' ]
ylabels = ['{}'.format(x) + 'M' for x in g.get_yticks()/1000000]
g.set_yticklabels(ylabels)
g.legend(newnames,loc=(2), fontsize=12)
plt.xticks(rotation = 45)
# add vertical line
#plt.vlines(x=16.5, ymin=0, ymax=5, color='gray', label='political event')
#trans = g.get_xaxis_transform()

# March 13, President Trump called a national emergency, due to COVID-19.
g.axvline("2020-03-13", color="gray", linestyle="dashed")
plt.text(0.25, 0.7, '13 March', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



#On April 3, President Trump fired Michael Atkinson, a senior official. 
g.axvline("2020-04-03", color="gray", linestyle="dashed")
plt.text(0.314, 0.7, '3 April', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


#BLM -- May 26, 2020
g.axvline("2020-05-26", color="gray", linestyle="dashed")
#plt.text("2020-05-26", .7, 'May 26', transform=trans)
plt.text(0.464, 0.7, '26 May', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #June 2 Blackout Tuesday
# g.axvline("2020-06-02", color="gray", linestyle="dashed")
# plt.text(0.51, 0.7, '2 June', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# July 3, Trump holds a campaign rally at taxpayer expense at Mount Rushmore, South Dakota, as social distancing is ignored and few mask are worn by the 7,500 participants
g.axvline("2020-07-03", color="gray", linestyle="dashed")
plt.text(0.58, 0.7, '3 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


#On July 22, it was announced that the Chinese consulate in Houston will close. 
#US Secretary of State Mike Pompeo stated the decision came after it was revealed that China was ?stealing? intellectual property. China condemned the decision, and various people can be seen burning papers outside the Houston consulate
# July 21, Twitter bans 7,000 QAnon accounts and limits 150,000 others.
g.axvline("2020-07-21", color="gray", linestyle="dashed")
plt.text(0.63, 0.7, '21 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



#11 August Joe Biden Announces Running Mate Kamala Harris, the first African American woman to serve in the role
g.axvline("2020-08-12", color="gray", linestyle="dashed")
#plt.text("2020-08-12", .7, 'August 12', transform=trans)
plt.text(0.7, 0.7, '11 August', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#g.axvline("2020-09-16", color="gray", linestyle="dashed")

#President Trump Appoint Amy Coney Barrett = September 26, 2020 
g.axvline("2020-09-26", color="gray", linestyle="dashed")
plt.text(0.83, 0.7, '26 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#September 27 - A New York Times report on President Trump's personal and business tax returns alleges years of tax avoidance and millions in debt and IRS penalties, among other allegations.

# #President Trump and presidential candidate Joe Biden had the first debate = September 29,2020 -- pro-biden
# g.axvline("2020-09-29", color="gray", linestyle="dashed")
# plt.text(0.86, 0.7, '29 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#President Trump and his wife, First Lady Melania, both tested positive for COVID = October 2, 2020 -- stance
g.axvline("2020-10-01", color="gray", linestyle="dashed")
plt.text(.87, .7, '2 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# October 22, the second 2020 presidential debate takes place in Nashville, Tennessee.
#October 26, Election officials in Pennsylvania report that one-fifth of the state's nine million voters have already cast their votes. 2.9 million mail-in requests were received by October 23.
g.axvline("2020-10-26", color="gray", linestyle="dashed")
plt.text(.92, .89, '26 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # #On October 25, Vice President Mike Pence have tested positive for coronavirus.
# g.axvline("2020-10-25", color="gray", linestyle="dashed")
# plt.text(.91, .83, '25 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #On October 27, police in Philadelphia fatally shot a black man = October 27
# g.axvline("2020-10-27", color="gray", linestyle="dashed")
# plt.text(.91, .83, '27 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# US 2020 election
g.axvline("2020-11-03", color="gray", linestyle="dashed")
plt.text(.964, .83, '3 November',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


plt.savefig("image/temporal_imp_stance.png", dpi=300, bbox_inches = "tight") 
plt.show()

###### Major events 

## collected from : https://www.infoplease.com/june-2020-current-events-us-news

March 13, President Trump called a national emergency, due to COVID-19.

On April 4, President Trump fired Michael Atkinson, a senior official. It is believed that Atkinson is the whistleblower that began the impeachment process of President Trump. 

blm protest start date = May 26, 2020 -- stance

Black Lives Matter Leads to Reforms = June 27, 2020 -- stance

July 2, 2020 Several states, including California and Indiana, postpone or reverse plans to reopen their economies, as the United States records 50,000 new cases of COVID-19—the largest one-day spike since the pandemic’s onset.

Joe Biden Announces Running Mate Kamala Harris who is a strong advocate for diversity and equality = August 11, 2020 

Protests Against Police Brutality Continue = August 24, 2020

President Trump to Appoint Amy Coney Barrett = September 26, 2020 

September 27 - A New York Times report on President Trump's personal and business tax returns alleges years of tax avoidance and millions in debt and IRS penalties, among other allegations.

President Trump and presidential candidate Joe Biden had the first debate = September 29,2020 -- pro-biden

President Trump and his wife, First Lady Melania, both tested positive for COVID = October 2, 2020 -- stance

Trump Administration Imposes Restrictions on H-1B Visa as a result of the high unemployment rate in the country = October 7, 2020 -- anti-trump

October 19 various states reported record-breaking numbers regarding early votes ahead of the November 3 election. Many states have seen a dramatic increase in the number of absentee ballots. States are also seeing a boom in US postal ballot requests. So far, registered Democrats have voted more than registered Republicans. 

October 22 - The second 2020 presidential debate takes place in Nashville, Tennessee.

On October 25, Vice President Mike Pence have tested positive for coronavirus.

On October 26, Election officials in Pennsylvania report that one-fifth of the state's nine million voters have already cast their votes. 2.9 million mail-in requests were received by October 23.[385]


On October 26, police in Philadelphia fatally shot a black man = October 27




## Timeseries of impressions based on issues

In [ ]:
temp

In [ ]:
temp['pred_issue'].value_counts().head(5)

In [ ]:
top5_issue = temp.loc[(temp.pred_issue == 10) | (temp.pred_issue == 3) | (temp.pred_issue == 4) | (temp.pred_issue == 11) | (temp.pred_issue == 1)]


top5_issue



In [ ]:
grp_temp_issue = temp.pivot_table('avg_imp', index='date', columns='pred_issue', aggfunc='sum')
#grp_temp_issue = top5_issue.pivot_table('avg_imp', index='date', columns='pred_issue', aggfunc='sum')

In [ ]:
grp_temp_issue

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  # use Seaborn styles
g = grp_temp_issue.plot(kind='line', figsize=(10,5), color=['coral', 'green', 'pink', 'blue', '#EDB120', 'cyan', 'brown', 'black', 'lime', 'magenta', 'red', 'gray','purple'])
#g = grp_temp_issue.plot(kind='line', figsize=(10,5), color=['cyan', 'green', 'orange', 'blue', 'red'])
plt.xlabel('Time', fontsize=12);
plt.ylabel('Impressions', fontsize=12);
newnames = ['abortion', 'covid', 'climate', 'criminal justice reform, race, law & order', 'economy & taxes', 'education', 'foreign policy', 'guns', 'healthcare', 'immigration', 'supreme court', 'terrorism', 'lgbtq']
#newnames = [ 'covid', 'criminal justice reform, race, law & order', 'economy & taxes',  'supreme court', 'terrorism']
ylabels = ['{}'.format(x) + 'M' for x in g.get_yticks()/1000000]
# g.set_yticklabels(ylabels)
# g.legend(newnames,loc=(1.01, 0), fontsize=12)
# plt.xticks(rotation = 45)
# #g.get_legend().remove()
# #plt.savefig("image/temporal_imp_issue.png", dpi=300, bbox_inches = "tight") 
# plt.show()

g.set_yticklabels(ylabels)
g.legend(newnames,loc=(2), fontsize=12)
plt.xticks(rotation = 45)
# add vertical line
#plt.vlines(x=16.5, ymin=0, ymax=5, color='gray', label='political event')
#trans = g.get_xaxis_transform()

# March 13, President Trump called a national emergency, due to COVID-19.
g.axvline("2020-03-13", color="gray", linestyle="dashed")
plt.text(0.25, 0.7, '13 March', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



#On April 3, President Trump fired Michael Atkinson, a senior official. 
g.axvline("2020-04-03", color="gray", linestyle="dashed")
plt.text(0.314, 0.7, '3 April', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


#BLM -- May 26, 2020
g.axvline("2020-05-26", color="gray", linestyle="dashed")
#plt.text("2020-05-26", .7, 'May 26', transform=trans)
plt.text(0.464, 0.7, '26 May', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #June 2 Blackout Tuesday
# g.axvline("2020-06-02", color="gray", linestyle="dashed")
# plt.text(0.51, 0.7, '2 June', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# July 3, Trump holds a campaign rally at taxpayer expense at Mount Rushmore, South Dakota, as social distancing is ignored and few mask are worn by the 7,500 participants
g.axvline("2020-07-03", color="gray", linestyle="dashed")
plt.text(0.58, 0.7, '3 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


#On July 22, it was announced that the Chinese consulate in Houston will close. 
#US Secretary of State Mike Pompeo stated the decision came after it was revealed that China was ?stealing? intellectual property. China condemned the decision, and various people can be seen burning papers outside the Houston consulate
# July 21, Twitter bans 7,000 QAnon accounts and limits 150,000 others.
g.axvline("2020-07-21", color="gray", linestyle="dashed")
plt.text(0.63, 0.7, '21 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



#11 August Joe Biden Announces Running Mate Kamala Harris, the first African American woman to serve in the role
g.axvline("2020-08-12", color="gray", linestyle="dashed")
#plt.text("2020-08-12", .7, 'August 12', transform=trans)
plt.text(0.7, 0.7, '11 August', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#g.axvline("2020-09-16", color="gray", linestyle="dashed")

#President Trump Appoint Amy Coney Barrett = September 26, 2020 
g.axvline("2020-09-26", color="gray", linestyle="dashed")
plt.text(0.83, 0.75, '26 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#September 27 - A New York Times report on President Trump's personal and business tax returns alleges years of tax avoidance and millions in debt and IRS penalties, among other allegations.

# #President Trump and presidential candidate Joe Biden had the first debate = September 29,2020 -- pro-biden
# g.axvline("2020-09-29", color="gray", linestyle="dashed")
# plt.text(0.86, 0.7, '29 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

#President Trump and his wife, First Lady Melania, both tested positive for COVID = October 2, 2020 -- stance
g.axvline("2020-10-01", color="gray", linestyle="dashed")
plt.text(.87, .7, '2 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# October 22, the second 2020 presidential debate takes place in Nashville, Tennessee.
#October 26, Election officials in Pennsylvania report that one-fifth of the state's nine million voters have already cast their votes. 2.9 million mail-in requests were received by October 23.
g.axvline("2020-10-26", color="gray", linestyle="dashed")
plt.text(.92, .89, '26 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # #On October 25, Vice President Mike Pence have tested positive for coronavirus.
# g.axvline("2020-10-25", color="gray", linestyle="dashed")
# plt.text(.91, .83, '25 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #On October 27, police in Philadelphia fatally shot a black man = October 27
# g.axvline("2020-10-27", color="gray", linestyle="dashed")
# plt.text(.91, .83, '27 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# US 2020 election
g.axvline("2020-11-03", color="gray", linestyle="dashed")
plt.text(.964, .83, '3 November',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)
g.legend(newnames,loc=(1.01, 0), fontsize=12)

plt.savefig("image/temporal_imp_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()


## poll data
### https://projects.fivethirtyeight.com/2020-general-data/presidential_poll_averages_2020.csv
### details here: https://github.com/fivethirtyeight/data/tree/master/election-forecasts-2020

#### pct_estimate = Polling average for the candidate listed in candidate_name on modeldate

In [ ]:
# ### Load poll data
poll = pd.read_csv( 'data/presidential_poll_averages_2020.csv',low_memory=False) 
poll #29085 rows × 6 columns

In [ ]:
poll['date']= pd.to_datetime(poll['modeldate']) #convert string to date-time
poll

In [ ]:
grp_poll = poll.pivot_table('pct_estimate', index='date', columns='candidate_name', aggfunc='sum')

In [ ]:
grp_poll

In [ ]:

new_poll = grp_poll.drop(columns=['Convention Bounce for Donald Trump', 'Convention Bounce for Joseph R. Biden Jr.'])
new_poll

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  # use Seaborn styles
g = new_poll.plot(kind='line', figsize=(10,5), color=['red', 'blue'])
plt.xlabel('Time', fontsize=12);
plt.ylabel('pct_estimate', fontsize=12);
newnames = ['Trump', 'Biden']
ylabels = ['{}'.format(x) + 'K' for x in g.get_yticks()/1000]
g.set_yticklabels(ylabels)
g.legend(newnames,loc=(2), fontsize=12)
plt.xticks(rotation = 45)
#g.get_legend().remove()
#plt.savefig("image/temporal_imp_issue.png", dpi=300, bbox_inches = "tight") 
plt.show()


In [ ]:
#replace nan with 0 in pandas
fb = grp_temp.fillna(0)
fb

In [ ]:
sum_trump = fb[1] + fb[2]
sum_biden = fb[0] + fb[3]
fb["ad_trump"] = sum_trump
fb["ad_biden"] = sum_biden
fb

In [ ]:
fb = fb.drop(columns=[0,1,2,3])
fb

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib.transforms as transforms
# sns.set()  # use Seaborn styles
# g = fb.plot(kind='line', figsize=(15,6), color=['red', 'blue'])
# plt.xlabel('Time', fontsize=12);
# plt.ylabel('Impressions', fontsize=12);
# newnames = ['pro-trump', 'pro-biden']
# ylabels = ['{}'.format(x) + 'M' for x in g.get_yticks()/1000000]
# g.set_yticklabels(ylabels)
# g.legend(newnames,loc=(2), fontsize=12)
# plt.xticks(rotation = 45)
# # add vertical line
# #plt.vlines(x=16.5, ymin=0, ymax=5, color='gray', label='political event')
# #trans = g.get_xaxis_transform()

# # March 13, President Trump called a national emergency, due to COVID-19.
# g.axvline("2020-03-13", color="gray", linestyle="dashed")
# plt.text(0.25, 0.7, '13 March', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



# #On April 3, President Trump fired Michael Atkinson, a senior official. 
# g.axvline("2020-04-03", color="gray", linestyle="dashed")
# plt.text(0.314, 0.7, '3 April', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# #BLM -- May 26, 2020
# g.axvline("2020-05-26", color="gray", linestyle="dashed")
# #plt.text("2020-05-26", .7, 'May 26', transform=trans)
# plt.text(0.464, 0.7, '26 May', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # #June 2 Blackout Tuesday
# # g.axvline("2020-06-02", color="gray", linestyle="dashed")
# # plt.text(0.51, 0.7, '2 June', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# # July 3, Trump holds a campaign rally at taxpayer expense at Mount Rushmore, South Dakota, as social distancing is ignored and few mask are worn by the 7,500 participants
# g.axvline("2020-07-03", color="gray", linestyle="dashed")
# plt.text(0.58, 0.7, '3 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# #On July 22, it was announced that the Chinese consulate in Houston will close. 
# #US Secretary of State Mike Pompeo stated the decision came after it was revealed that China was ?stealing? intellectual property. China condemned the decision, and various people can be seen burning papers outside the Houston consulate
# # July 21, Twitter bans 7,000 QAnon accounts and limits 150,000 others.
# g.axvline("2020-07-21", color="gray", linestyle="dashed")
# plt.text(0.63, 0.7, '21 July', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)



# #11 August Joe Biden Announces Running Mate Kamala Harris, the first African American woman to serve in the role
# g.axvline("2020-08-11", color="gray", linestyle="dashed")
# #plt.text("2020-08-12", .7, 'August 12', transform=trans)
# plt.text(0.7, 0.7, '11 August', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #g.axvline("2020-09-16", color="gray", linestyle="dashed")

# #President Trump Appoint Amy Coney Barrett = September 26, 2020 
# g.axvline("2020-09-26", color="gray", linestyle="dashed")
# plt.text(0.83, 0.7, '26 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #September 27 - A New York Times report on President Trump's personal and business tax returns alleges years of tax avoidance and millions in debt and IRS penalties, among other allegations.

# # #President Trump and presidential candidate Joe Biden had the first debate = September 29,2020 -- pro-biden
# # g.axvline("2020-09-29", color="gray", linestyle="dashed")
# # plt.text(0.86, 0.7, '29 September', rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# #President Trump and his wife, First Lady Melania, both tested positive for COVID = October 2, 2020 -- stance
# g.axvline("2020-10-01", color="gray", linestyle="dashed")
# plt.text(.87, .7, '2 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # October 22, the second 2020 presidential debate takes place in Nashville, Tennessee.
# #October 26, Election officials in Pennsylvania report that one-fifth of the state's nine million voters have already cast their votes. 2.9 million mail-in requests were received by October 23.
# g.axvline("2020-10-26", color="gray", linestyle="dashed")
# plt.text(.92, .89, '26 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # # #On October 25, Vice President Mike Pence have tested positive for coronavirus.
# # g.axvline("2020-10-25", color="gray", linestyle="dashed")
# # plt.text(.91, .83, '25 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)

# # #On October 27, police in Philadelphia fatally shot a black man = October 27
# # g.axvline("2020-10-27", color="gray", linestyle="dashed")
# # plt.text(.91, .83, '27 October',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# # US 2020 election
# g.axvline("2020-11-03", color="gray", linestyle="dashed")
# plt.text(.964, .83, '3 November',  rotation = 90, horizontalalignment='center', verticalalignment='center', transform=g.transAxes)


# #plt.savefig("image/temporal_imp_stance.png", dpi=300, bbox_inches = "tight") 
# plt.show()

In [ ]:
fb_idx = fb.index
poll_idx = new_poll.index

In [ ]:
fb['date'] = fb_idx.values
new_poll['date'] = poll_idx.values

In [ ]:
new_poll = new_poll.reset_index(drop=True)
fb = fb.reset_index(drop=True) 

In [ ]:
new_poll

In [ ]:
fb

In [ ]:
#compare two dataframes based on certain column values and remove rest

new_fb = fb.merge(new_poll,how='left').dropna()
new_fb = new_fb.reset_index(drop = True)


In [ ]:
new_fb

In [ ]:
ad_pol = new_fb.rename({'Donald Trump': 'poll_trump', 'Joseph R. Biden Jr.': 'poll_biden'}, axis='columns')


In [ ]:
ad_pol

## Granger Causality

### Do AdsImpressions Granger-cause Poll count ??
#### Null hypothesis (H0): AdsImpressions do not Granger cause Poll count
#### Alternative Hypothesis (HA) : AdsImpressions Granger-cause Poll count




In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests



In [ ]:
def do_granger(ac, pc, lags):
    #The data for test whether the time series in the second column Granger causes the time series in the first column.
    input = np.stack((pc, ac), 1) #Do AdsImpressions Granger-cause Poll count ?
    #input = np.stack((ac, pc), 1) #other way..Do Poll counts Granger-cause AdsImpressions ?
   
    gc = grangercausalitytests(input, maxlag = lags ) #outputs all lags
    p_val = []
    F = []
    for key, value in gc.items() :
        #print (key,value)
        #print ("value[0]", value[0])
#         p_val = []
        for k, v in value[0].items() :
            #print("k,v", k,v)
            #print(k)
            #print(v)
            #print(v[0])
            #print(type(v[0]))
            if k == 'ssr_ftest':
                #print(v, type (v))
                #print('p-value', v[1])
                if v[0] < 0.00001:
                    F.append(float(0.0001))
                else:    
                    F.append(v[0])
                p_val.append(v[1])

    print(p_val, F)
                
    return p_val, F    

In [ ]:
#######......Granger ............

#For Trump ad and poll
act = np.array(ad_pol.ad_trump)
pct = np.array(ad_pol.poll_trump)

lags = 25
p_valt1, FT1 = do_granger(act, pct, lags)

#For Biden ad and poll
acb = np.array(ad_pol.ad_biden)
pcb = np.array(ad_pol.poll_biden)
p_valb1, FB1 = do_granger(acb, pcb, lags)


    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(FT1, '.-', color='red',  label='pro-trump, anti-biden')
plt.plot(FB1, '.-', color='blue', label='pro-biden, anti-trump')
plt.xlabel('Lags', fontsize=12)
plt.ylabel('SSR F-test', fontsize=12)
plt.legend(loc=(4), fontsize=12)
#plt.legend()

plt.savefig("image/gc2.png",  bbox_inches = "tight") 
plt.show()

### Now repeat the Granger causality test in the opposite direction.

### Do Poll counts Granger-cause  AdsImpressions  ??
#### Null Hypothesis (H0) : Poll counts do not Granger-cause  AdsImpressions
#### Alternative Hypothesis (HA) : Poll counts Granger-cause  AdsImpressions

In [ ]:
def do_grangerAlt(ac, pc, lags):
    #The data for test whether the time series in the second column Granger causes the time series in the first column.
    input = np.stack((ac, pc), 1) #other way..Do Poll counts Granger-cause AdsImpressions ?
   
    gc = grangercausalitytests(input, maxlag = lags ) #outputs all lags
    p_val = []
    F = []
    for key, value in gc.items() :
        #print (key,value)
        #print ("value[0]", value[0])
#         p_val = []
        for k, v in value[0].items() :
            #print("k,v", k,v)
            #print(k)
            #print(v)
            #print(v[0])
            #print(type(v[0]))
            if k == 'ssr_ftest':
                #print(v, type (v))
                #print('p-value', v[1])
                if v[0] < 0.00001:
                    F.append(float(0.0001))
                else:    
                    F.append(v[0])
                p_val.append(v[1])

    print(p_val, F)
                
    return p_val, F    

In [ ]:
#For Trump ad and poll
act = np.array(ad_pol.ad_trump)
pct = np.array(ad_pol.poll_trump)

lags = 25
p_valt2, FT2 = do_grangerAlt(act, pct, lags)

#For Biden ad and poll
acb = np.array(ad_pol.ad_biden)
pcb = np.array(ad_pol.poll_biden)
p_valb2, FB2 = do_grangerAlt(acb, pcb, lags)


    

In [ ]:
import matplotlib.pyplot as plt
markers_xT = [0, 1, 2, 3, 4, 24]
for i in markers_xT:
    plt.plot(i, FT2[i], 'D', color = 'black')
    
markers_xB = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 23, 24]
for i in markers_xB:
    plt.plot(i, FB2[i], 'D', color = 'black')
    
plt.plot(FT2, '.-', color='red', label='pro-trump, anti-biden')
plt.plot(FB2, '.-', color='blue', label='pro-biden, anti-trump')
plt.xlabel('Lags', fontsize=12)
plt.ylabel('SSR F-test', fontsize=12)
plt.legend(loc=(1), fontsize=12)
#g.legend(newnames,loc=(2), fontsize=12)
plt.savefig("image/gc1.png",  bbox_inches = "tight") 
plt.show()